In [1]:
from functools import partial
from sqlite3 import connect

import astroplan as ap
from astropy.coordinates import EarthLocation
import astropy.units as u
from icecream import ic, install

import astropaul.targetlistcreator as tlc
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

install() # icecream will now work in imported packages

%load_ext autoreload
%autoreload 2


In [2]:
from astropy.time import Time

foo = ap.Observer(EarthLocation("-78:41:34.92", "37:52:41.35", 566 * u.m), timezone="utc", name="Fan Mountain")
bar = ap.Observer.at_site("APO")
foo.sun_rise_time(Time("2025-04-16 19:00", format="iso"), which="next").iso
foo, bar

(<Observer: name='Fan Mountain',
     location (lon, lat, el)=(-78.69303333333335 deg, 37.878152777777785 deg, 565.9999999989083 m),
     timezone=<UTC>>,
 <Observer: name='APO',
     location (lon, lat, el)=(-105.82000000000002 deg, 32.78000000000001 deg, 2797.9999999996007 m),
     timezone=<UTC>>)

In [ ]:
conn = connect("../../django/TargetDB/db.sqlite3")

# session = tlc.ObservingSession(ap.Observer.at_site("APO"))
session = tlc.ObservingSession(ap.Observer(EarthLocation("-78:41:34.92", "37:52:41.35", 566 * u.m), name="Fan Mountain"))
session.add_day_range("2025-05-15", "2025-08-21")

phase_event_defs = [
    ph.PhaseEventDef("Not in Eclipse", partial(ph.calc_time_of_gress, ingress=False)),
    ph.PhaseEventDef("Eclipse", partial(ph.calc_time_of_gress, ingress=True)),
]

wide_bright_qeb = [
    "TIC 278465736",
    "TIC 470710327",
    "TIC 367448265",
    "TIC 376606423",
    "TIC 317863971",
    "TIC 159089190",
    "TIC 70664819",
    "TIC 286470992",
    "TIC 78568780",
    "TIC 200094011",
]

min_altitude = 35 * u.deg

name = "Fan Mountain SIDE Summer 2025"
creator = tlc.TargetListCreator(name=name, connection=conn)
creator.steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: (df["Target Name"].isin(wide_bright_qeb))),
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="TESS "),
    partial(
        tlc.add_observability,
        observing_session=session,
        calc_moon_distance=True,
        observability_threshold=(min_altitude, 80 * u.deg),
    ),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Any Night"])),
    tlc.add_ephemerides,
    partial(tlc.add_phase_events, observing_session=session, phase_event_defs=phase_event_defs),
    tlc.add_speckle,
    # partial(tlc.add_speckle_phase, phase_event_defs=phase_event_defs),
    partial(tlc.add_side_status, phase_event_defs=phase_event_defs),
]
tl = creator.calculate(verbose=False)

# html.render_observing_pages(tl, None, {}, "html")
print(tl.summarize())

C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:426: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  answer.target_list[max_alt_column] = segment_max_alts
C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:437: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  answer.target_list[moon_dist_column] = dist
C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:414: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whic

ValueError: You are trying to merge on object and int64 columns for key 'Target Name'. If you wish to proceed you should use pd.concat

: 

In [ ]:
tl.other_lists["SIDE Observations"]

,Speckle Session,JD Mid,UTC Mid,System,Member,SIDE Type
Target Name,,,,,,
TIC 278465736,2303,2.460459e+06,2024-05-28 09:06:00,A,a,Exact
TIC 278465736,2325,2.460459e+06,2024-05-28 10:35:30,A,a,Exact
TIC 278465736,2581,2.460578e+06,2024-09-24 07:04:00,A,b,Exact
TIC 278465736,2817,2.460631e+06,2024-11-16 01:01:30,A,b,Exact
TIC 278465736,2825,2.460631e+06,2024-11-16 02:22:30,A,b,Exact
TIC 278465736,2831,2.460631e+06,2024-11-16 03:28:00,A,b,Exact
TIC 278465736,2817,2.460631e+06,2024-11-16 01:01:30,B,a,Exact
TIC 278465736,2825,2.460631e+06,2024-11-16 02:22:30,B,a,Exact
TIC 278465736,2831,2.460631e+06,2024-11-16 03:28:00,B,a,Exact


In [ ]:
altitude_categories = [
    ((-90, min_altitude.value), 0),
    ((min_altitude.value, 90), 1),
]

pl = pr.PriorityList(tl, session, interval=30 * u.min)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
pr.prioritize_phase_sequence(pl, ["Eclipse"], "Eclipse", True, True, True)
pr.prioritize_side_observation(pl, side_state="Eclipse")
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.0)
pl.categorize_priorities(bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"])

html.render_observing_pages(tl, pl, {}, f"../../Observing Files/{tl.name}")
# pl.categorical_priorities[0]